In [1]:
import polars as pl
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly import subplots
import mlxtend as mlx
import numpy as np
import pandas as pd
import statsmodels
from IPython.display import display, HTML

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 50

# Convert scientific notation to plain numbers
pd.options.display.float_format = '{:.2f}'.format

def display_side_by_side(*args):
    html_str = '<div style="display: flex; justify-content: flex-start;">'
    for df in args:
        if isinstance(df, pd.Series):
            df = df.to_frame()
        html_str += f'<div style="flex: 1; padding: 0;">{df.to_html(index=True)}</div>'
    html_str += '</div>'
    
    display(HTML(html_str))

In [2]:
#Take an initial look at the data's first and last rows, and get an overview of its shape

df = pd.read_csv('online_retail.csv')
display_side_by_side(df.head())
display_side_by_side(df.tail())


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.00,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.00,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.00,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.00,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.00,France


In [3]:
print('-----------------')
print(f'Dataframe contains {df.shape[1]} columns and {df.shape[0]} rows')

-----------------
Dataframe contains 8 columns and 541909 rows


In [4]:
#Add a TotalPrice column that allows us to factor in both price and quantity ordered
df['TotalPrice'] = df['UnitPrice'] * df['Quantity']

#Display all of the dtypes in the dataframe
dtypes_df = pd.DataFrame(df.dtypes).reset_index()
dtypes_df.columns = ['column name', 'dtype']
print(f'Unique dtypes in this DataFrame:{set(dtypes_df["dtype"].values)}')

print('-------------------------------')


Unique dtypes in this DataFrame:{dtype('O'), dtype('float64'), dtype('int64')}
-------------------------------


In [ ]:
#Display which columns follow under what dtype, separated by kind
object_df, float_df, int_df= dtypes_df[dtypes_df['dtype']== 'object'], dtypes_df[dtypes_df['dtype']== 'float64'], dtypes_df[dtypes_df['dtype']== 'int64']
display_side_by_side(object_df, float_df,  int_df)

,column name,dtype
0,InvoiceNo,object
1,StockCode,object
2,Description,object
4,InvoiceDate,object
7,Country,object
,column name,dtype
5,UnitPrice,float64
6,CustomerID,float64
8,TotalPrice,float64
,column name,dtype


In [6]:
#Check to see if we have any NaNs, Nulls or Duplicates
print(f'DF has NaNs: {df.isna().any().all()}')
print(f'DF has Null: {df.isnull().any().all()}')
print(f'DF has Duplicates: {df.duplicated().any().all()}')

print('-----------')

#Check to see if our DF contains duplicate customers or if they are all unique
print(f"Rows containing purchases from the same Customer ID: {df.duplicated(subset='CustomerID').any().all()}")
print(f"Number of Unique Customer IDs:{len(np.unique(df['CustomerID']))}")

DF has NaNs: False
DF has Null: False
DF has Duplicates: True
-----------
Rows containing purchases from the same Customer ID: True
Number of Unique Customer IDs:4373


In [7]:
#let's get a quick look at some of the Duplicates reported
df[df.duplicated()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.00,United Kingdom,1.25
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.00,United Kingdom,2.10
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.00,United Kingdom,2.95
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.00,United Kingdom,4.95
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.00,United Kingdom,2.95
...,...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.00,United Kingdom,0.39
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.00,United Kingdom,2.49
541692,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.00,United Kingdom,1.95
541699,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.00,United Kingdom,2.10


In [8]:
#Let's examine a transaction listed above that contains multiple duplicates
invoice_check = df[(df['InvoiceNo'] == '536409')]

#Check the shape first to see total order length
print(invoice_check.shape)

#Get the codes for all of the duplicated stock, and display all the transaction lines containing said codes within the order
invoice_dupe_stocks = invoice_check[invoice_check.duplicated(subset='StockCode')]
invoice_dupe_list = list(invoice_dupe_stocks['StockCode'])
invoice_check[invoice_check['StockCode'].isin(invoice_dupe_list)].sort_values('StockCode')

(58, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.00,United Kingdom,1.25
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.00,United Kingdom,1.25
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.00,United Kingdom,4.95
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.00,United Kingdom,4.95
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.00,United Kingdom,2.10
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.00,United Kingdom,2.10
521,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.00,United Kingdom,2.95
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.00,United Kingdom,2.95
491,536409,85116,BLACK CANDELABRA T-LIGHT HOLDER,1,2010-12-01 11:45:00,2.10,17908.00,United Kingdom,2.10
502,536409,85116,BLACK CANDELABRA T-LIGHT HOLDER,5,2010-12-01 11:45:00,2.10,17908.00,United Kingdom,10.50


In [9]:
#So 13 duplicated StockCode lines out of 58 total items, and some of these subset duplicates have different quantities ordered, so we can assume that while they are duplicates, we don't necessarily need to remove them and can assume that they are legitimate transaction lines

In [10]:
#The variables table for this dataset has mentioned that a 'C' in the invoice means the transaction was cancelled - let's get an idea of how many there are
print(f'Number of rows containing cancelled transactions: {len(df[df["InvoiceNo"].str.contains("C")])}')
print('----------------------------------')
#Since this is a sizeable amount of data, let's filter them out and focus on analysing completed orders moving forward
completed_df = df[~df['InvoiceNo'].str.contains('C')]
print(f'Original Dataframe length :  {len(df)}')
print(f'Dataframe length after removing cancelled transactions: {len(completed_df)}')

Number of rows containing cancelled transactions: 9288
----------------------------------
Original Dataframe length :  541909
Dataframe length after removing cancelled transactions: 532621


In [11]:
from statsmodels.stats import descriptivestats


#Get an overview of our other numerical columns
stats_df = descriptivestats.describe(completed_df).round(1)
display_side_by_side(stats_df[:15].round(1), stats_df[15:].round(1))

,Quantity,UnitPrice,CustomerID,TotalPrice
nobs,532621.00,532621.00,532621.00,532621.00
missing,0.00,0.00,134697.00,0.00
mean,10.20,3.80,15294.30,20.00
std_err,0.20,0.10,2.70,0.40
upper_ci,10.70,4.00,15299.60,20.70
lower_ci,9.80,3.70,15289.00,19.30
std,159.60,41.80,1713.20,270.60
iqr,9.00,2.90,2826.00,14.00
iqr_normal,6.70,2.10,2094.90,10.30
mad,11.50,3.20,1478.50,19.20


In [ ]:
#Identify what negative quantities and negative unit prices mean with regards to completed orders, which can be seen from the min values
negative_qty = completed_df[completed_df['Quantity']<0]
display_side_by_side(negative_qty.head(3))
display_side_by_side(negative_qty.tail(3))

negative_price = completed_df[completed_df['UnitPrice']<0]
display_side_by_side(negative_price)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom,-0.0
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom,-0.0
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom,-0.0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom,-0.0
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom,-0.0
538919,581422,23169,smashed,-235,2011-12-08 15:24:00,0.0,NaN,United Kingdom,-0.0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06


In [ ]:
#It looks like negative quantities are most likely tied to returns or other internal transactions that serve as manual inventory adjustments to reflect shrinkage
#Negative price on the other hand is solely related to what looks like a tax-specific adjustment

#Let's also filter both of these out to focus only on positive transactions
completed_positive_df = completed_df[ (completed_df['Quantity'] > 0) & (completed_df['UnitPrice'] > 0)]

In [ ]:
#Since our max Quantity and Unit Price also seem unusual especially when comapred to the 99% percentile values, let's inspect those as well
qty_sorted = completed_positive_df.sort_values('Quantity', ascending=False)
unit_price_sorted = completed_positive_df.sort_values('UnitPrice', ascending=False)

display_side_by_side(qty_sorted.head())
display_side_by_side(unit_price_sorted[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.60
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.60
421632,573008,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,4800,2011-10-27 12:26:00,0.21,12901.0,United Kingdom,1008.00
206121,554868,22197,SMALL POPCORN HOLDER,4300,2011-05-27 10:52:00,0.72,13135.0,United Kingdom,3096.00
97432,544612,22053,EMPIRE DESIGN ROSETTE,3906,2011-02-22 10:43:00,0.82,18087.0,United Kingdom,3202.92


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
15017,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,11062.06
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,8142.75
297723,562955,DOT,DOTCOM POSTAGE,1,2011-08-11 10:14:00,4505.17,NaN,United Kingdom,4505.17
268028,560373,M,Manual,1,2011-07-18 12:30:00,4287.63,NaN,United Kingdom,4287.63


In [ ]:
#The high quantity orders look to be legitimate wholesale orders but the high unit price items seems to be related to non-common stock charges, let's see what these are - noting that they all begin with a letter instead of the common 5-6 alphanumeric codes assigned to regular stock
import re

letter_stock_df = completed_positive_df[completed_positive_df['StockCode'].str.match(r'^[A-Z]')]

#Let's display all these codes, the descriptions, the counts, and also generate a list so we can filter them out if needed as well
letter_stock_counts_df = pd.DataFrame(letter_stock_df[['StockCode', 'Description']].groupby('StockCode').value_counts())
letter_stock_reset = letter_stock_counts_df.reset_index()
letter_stock_list = list(letter_stock_reset['StockCode'])

print(letter_stock_list)
letter_stock_counts_df

['AMAZONFEE', 'B', 'BANK CHARGES', 'C2', 'DCGS0003', 'DCGS0004', 'DCGS0069', 'DCGS0070', 'DCGS0076', 'DCGSSBOY', 'DCGSSGIRL', 'DOT', 'M', 'PADS', 'POST', 'S']


,,count
StockCode,Description,
AMAZONFEE,AMAZON FEE,2
B,Adjust bad debt,1
BANK CHARGES,Bank Charges,12
C2,CARRIAGE,141
DCGS0003,BOXED GLASS ASHTRAY,4
DCGS0004,HAYNES CAMPER SHOULDER BAG,1
DCGS0069,OOH LA LA DOGS COLLAR,1
DCGS0070,CAMOUFLAGE DOG COLLAR,1
DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,2


In [ ]:
#As most of these seem to be related to miscellaneous fees and charges, we will exclude them as well and save the filtered transaction list to a sales_csv file, which we will further analyze in a different notebook
stock_sales = completed_positive_df[~completed_positive_df['StockCode'].isin(letter_stock_list)]
stock_sales.to_csv('online_retail_pos_stock.csv', index=False)